# **BLOQUE 1: CONFIGURACIÓN E IMPORTACIÓN DE LIBRERIAS:**

In [1]:
# --- BLOQUE 1: CONFIGURACIÓN E IMPORTACIÓN ---
from google.colab import drive
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de estilo para gráficos (le da un toque más profesional)
sns.set_theme(style="whitegrid", palette="muted")

# Montamos Drive
drive.mount('/content/drive')

# --- CONSTANTES DEL PROYECTO ---
# Definimos aquí las variables clave para facilitar cambios futuros
AGLOMERADOS_INTERES = [34, 12]  # 34: Mar del Plata, 12: Corrientes
RUTA_CARPETA_EPH = '/content/drive/MyDrive/ColabNotebooks/Trimestres_anio_archivoTxt'
RUTA_ARCHIVO_IPC = '/content/drive/MyDrive/ColabNotebooks/sh_ipc_aperturas.xlsx'

print("✅ Entorno configurado correctamente.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Entorno configurado correctamente.


# **BLOQUE 2: CARGA DE DATOS EPH**

In [2]:
# --- BLOQUE 2: CARGA DE DATOS EPH ---

def cargar_bases_eph(ruta_carpeta, aglomerados_filtro):
    """
    Recorre la carpeta especificada, carga los archivos .txt y concatena
    solo los registros que coinciden con los aglomerados solicitados.
    """
    lista_dfs = []

    # Verificamos que la ruta exista
    if not os.path.exists(ruta_carpeta):
        print(f"❌ Error: No se encuentra la ruta {ruta_carpeta}")
        return None

    archivos = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith('.txt')]
    print(f"📂 Se encontraron {len(archivos)} archivos de bases EPH.")

    columnas_clave = ['ano4', 'trimestre', 'aglomerado', 'estado', 'pondera', 'p21', 'pp3e_tot', 'ch04', 'ch06', 'nived']

    for archivo in archivos:
        ruta_completa = os.path.join(ruta_carpeta, archivo)
        try:
            # Leemos el archivo (el separador suele ser ; o , dependiendo del año)
            # Probamos primero con ;
            df_temp = pd.read_csv(ruta_completa, sep=';', encoding='latin-1', low_memory=False)

            # Estandarizamos columnas a minúsculas para evitar errores (ANO4 vs ano4)
            df_temp.columns = df_temp.columns.str.lower()

            # Si falló el separador y cargó todo en una columna, reintentamos con coma
            if df_temp.shape[1] < 5:
                 df_temp = pd.read_csv(ruta_completa, sep=',', encoding='latin-1', low_memory=False)
                 df_temp.columns = df_temp.columns.str.lower()

            # FILTRO 1: Solo los aglomerados que nos interesan (ahorra memoria)
            # Convertimos a numérico para asegurar coincidencia
            df_temp['aglomerado'] = pd.to_numeric(df_temp['aglomerado'], errors='coerce')
            df_filtrado = df_temp[df_temp['aglomerado'].isin(aglomerados_filtro)].copy()

            if not df_filtrado.empty:
                # Creamos la columna PERIODO para series de tiempo
                df_filtrado['periodo'] = df_filtrado['ano4'].astype(str) + 'Q' + df_filtrado['trimestre'].astype(str)
                lista_dfs.append(df_filtrado)
                print(f"  -> Cargado: {archivo} ({len(df_filtrado)} registros de interés)")

        except Exception as e:
            print(f"⚠️ Advertencia: No se pudo leer {archivo}. Error: {e}")

    if lista_dfs:
        base_final = pd.concat(lista_dfs, ignore_index=True)
        print("-" * 50)
        print(f"✅ Carga completa. Total de registros acumulados: {len(base_final)}")
        return base_final
    else:
        print("❌ No se pudieron cargar datos.")
        return None

# Ejecutamos la función
base_eph = cargar_bases_eph(RUTA_CARPETA_EPH, AGLOMERADOS_INTERES)

# Vistazo rápido
if base_eph is not None:
    display(base_eph.head())
    print("Aglomerados cargados:", base_eph['aglomerado'].unique())

📂 Se encontraron 37 archivos de bases EPH.
  -> Cargado: usu_individual_T216.txt (2855 registros de interés)
  -> Cargado: usu_Individual_T220.txt (1897 registros de interés)
  -> Cargado: usu_individual_T221.txt (2317 registros de interés)
  -> Cargado: usu_individual_T223.txt (2139 registros de interés)
  -> Cargado: usu_individual_T224.txt (2012 registros de interés)
  -> Cargado: usu_individual_T225.txt (1915 registros de interés)
  -> Cargado: usu_individual_T222.txt (2176 registros de interés)
  -> Cargado: usu_individual_T217.txt (2638 registros de interés)
  -> Cargado: usu_individual_T218.txt (2598 registros de interés)
  -> Cargado: usu_individual_T219.txt (2785 registros de interés)
  -> Cargado: usu_individual_t416.txt (2805 registros de interés)
  -> Cargado: usu_individual_T424.txt (1956 registros de interés)
  -> Cargado: usu_individual_T423.txt (2025 registros de interés)
  -> Cargado: usu_individual_T421.txt (2235 registros de interés)
  -> Cargado: usu_individual_T419

,codusu,ano4,trimestre,nro_hogar,componente,h15,region,mas_500,aglomerado,pondera,...,v22_01_m,v22_02_m,v22_03_m,p_deccf,p_rdeccf,p_gdeccf,p_pdeccf,p_ideccf,p_adeccf,unnamed: 177
0,TQRMNORPVHLMKQCDEIKAH00485668,2016,2,1,3,1,43,S,34,522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TQRMNORPVHLMKQCDEIKAH00485668,2016,2,1,4,1,43,S,34,522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TQRMNOQYQHLMKUCDEIKAH00485679,2016,2,1,1,1,43,S,34,500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TQRMNOQYQHLMKUCDEIKAH00485679,2016,2,1,2,1,43,S,34,500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TQRMNOQVWHKOLPCDEIKAH00485617,2016,2,1,1,1,43,S,34,498,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Aglomerados cargados: [34 12]


# **BLOQUE 3: PREPARACIÓN DE DATOS DE INFLACIÓN (IPC)**

In [4]:
# --- BLOQUE 3: PREPARACIÓN DE DATOS DE INFLACIÓN (IPC) ---

def procesar_ipc_indec(ruta_excel):
    print("🔄 Procesando archivo de inflación...")
    try:
        # Cargamos la hoja "Índices aperturas". El header suele estar en la fila 5 (índice 5 si contamos desde 0)
        df_ipc = pd.read_excel(ruta_excel, sheet_name='Índices aperturas', header=5)

        # Detectamos columnas de fecha dinámicamente
        cols_fecha = [c for c in df_ipc.columns if isinstance(c, (pd.Timestamp, str)) and str(c).startswith('20')]

        # Transformamos de formato ancho a largo (Melt)
        df_melt = df_ipc.melt(id_vars=[df_ipc.columns[0]], value_vars=cols_fecha, var_name='fecha', value_name='indice')

        # Limpieza de fechas
        df_melt['fecha'] = pd.to_datetime(df_melt['fecha'], errors='coerce')
        df_melt['periodo'] = df_melt['fecha'].dt.year.astype(str) + 'Q' + df_melt['fecha'].dt.quarter.astype(str)

        # Agrupamos por trimestre (promedio de los 3 meses del trimestre)
        # Filtramos por "Nivel general" si es posible, o tomamos el promedio global
        ipc_trimestral = df_melt.groupby('periodo')['indice'].mean().reset_index()

        # Calculamos el FACTOR DE AJUSTE (Llevar todo a valores del último trimestre disponible)
        ultimo_indice = ipc_trimestral['indice'].iloc[-1]
        ipc_trimestral['factor_ajuste'] = ultimo_indice / ipc_trimestral['indice']

        print("✅ Tabla de coeficientes IPC creada exitosamente.")
        return ipc_trimestral

    except Exception as e:
        print(f"❌ Error al procesar IPC: {e}")
        return None

# Ejecutamos
df_factores_ipc = procesar_ipc_indec(RUTA_ARCHIVO_IPC)
if df_factores_ipc is not None:
    display(df_factores_ipc.tail())

🔄 Procesando archivo de inflación...
❌ Error al procesar IPC: single positional indexer is out-of-bounds
